In [ ]:
import tensorflow as tf
import pickle
import import_ipynb
from model import Model
from utils import build_dict, build_dataset, batch_iter

def ml_headline(q2,args1):
    
    embedding_size=300
    num_hidden = 300
    num_layers = 3
    learning_rate = 0.001
    beam_width = 10
    keep_prob = 0.8
    glove = True
    batch_size=128
    num_epochs=15
    while True:
        print("Loading dictionary...")
        word_dict, reversed_dict, article_max_len, summary_max_len = build_dict("valid", False)

        #print("content",content)
        #queue get하기 (data정보 가져오기)
        data=q2.get()
        content=normalize(data[1])
        x = [normalize(content)]
        x = [word_dict.get(w, word_dict["<unk>"]) for w in x]
        x = [d[:120] for d in x]
        valid_x = [d + (article_max_len - len(d)) * [word_dict["<padding>"]] for d in x]
        valid_x_len = [len([y for y in x if y != 0]) for x in valid_x]

        with tf.Session() as sess:
        print("Loading saved model...")
        model = Model(reversed_dict, article_max_len, summary_max_len, embedding_size, num_hidden, num_layers, learning_rate, beam_width, keep_prob, glove, forward_only=True)
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state("./saved_model/")
        saver.restore(sess, ckpt.model_checkpoint_path)

        batches = batch_iter(valid_x, [0] * len(valid_x), batch_size, 1)

        for batch_x, _ in batches:
            batch_x_len = [len([y for y in x if y != 0]) for x in batch_x]

            valid_feed_dict = {
                model.batch_size: len(batch_x),
                model.X: batch_x,
                model.X_len: batch_x_len,
            }

            prediction = sess.run(model.prediction, feed_dict=valid_feed_dict)
            prediction_output = [[reversed_dict[y] for y in x] for x in prediction[:, 0, :]]
            summary=list()
            predict=""
            for word in prediction_output:
                if word == "</s>":
                    break
                if word not in summary:
                    summary.append(word)
                predict=" ".join(summary)
            data.append(predict)
            db_update(data)

In [ ]:
from lexrankr import LexRank
import random
import requests
import json
from apscheduler.jobstores.base import JobLookupError
from apscheduler.schedulers.background import BackgroundScheduler
import math
import copy
import time
from multiprocessing import Process, Queue
import pandas as pd

#-*- coding: utf-8 -*-
def random_four_news(q1):
    print("random_four_news start")
    articles = requests.get(url="http://34.84.147.192:8000/news/articles/?format=json&limit=1").json()
    count = articles['count']
    print(count)

    articles = requests.get(url="http://34.84.147.192:8000/news/articles/?format=json&limit=" + str(count)).json()
    d = pd.DataFrame(articles['results'])

    df = pd.DataFrame(columns=['news_id', 'summary', 'cluster_id'])
    df = d[['news_id', 'summary', 'cluster_id']]
    cluster = requests.get(url="http://34.84.147.192:8000/news/clusters/").json()
    #cluster = df['cluster_id']
    #print(cluster)
    cluster_id = list([i["cluster_id"] for i in cluster if i["cluster_id"] != '07f269a8-3ae6-4994-abfd-e2cb2d4633f3'])
    print(len(cluster_id))
    #global cluster_len
    #cluster_len=len(cluster_id)
    df_summary=[]
    tmp=[]
    i=0
    #먼저 cluster_id에 해당되는 애들 dict에 넣기
    for cluster_id_1 in cluster_id:
        print("x",cluster_id_1)
        df2 = df[df["cluster_id"] == cluster_id_1]
        df_news = df2['news_id'].tolist()
        news_id = random.sample(df_news, 4)
        for news in news_id:
            df3 = df[df["news_id"] == news]
            df_summary += df3['summary'].tolist()
        
        tmp.append(cluster_id_1)
        tmp.append(" ".join(df_summary))
        #print("TTTTTTTtmp",tmp)
        q1.put(tmp)
        tmp=[]
        df_summary=[]



def summary(q1,q2):
    while True:
        print("smry start")
        multi_summary = q1.get()
        #print("q1:",q1.qsize())
        #print("df",multi_summary)
        #print(multi_summary)
        lexrank = LexRank()
        #print(multi_summary[0])
        lexrank.summarize(multi_summary[1])  # data (본문)가져와서 요약
        summaries = lexrank.probe(3)  # 3줄요약, summaries 타입은 list
        summaries = '. '.join(summaries)+'.'
        print("multi-summary= ",summaries)
        multi_summary[1] = summaries
        #ml_headline(data,"test")
        q2.put(multi_summary) # db에 저장되어야 하는 최종 결과
            # 입력데이터가 이상한 값이어서 요약문이 안나올 때 에러처리 #입력데이터가 None으로 들어올때 에러처리
        


def db_update(data):
    URL = "http://34.84.147.192:8000/news/"
    clusterId = data[0]
    summaryy=data[1].split('. ')
    summaryy = "\n\n".join(summaryy)
    print(data)
    cluster = {
        "cluster_headline": data[2],
        "cluster_summary": summaryy
    }
    res = requests.put(url = URL + 'clusters/' + clusterId + '/',data=cluster)


In [ ]:
if __name__ == "__main__":
    q1 = Queue()
    q2 = Queue()
    sched = BackgroundScheduler()

    sched.start()
    sched.add_job(random_four_news, 'cron', hour='1', id="s1", args=[df, cluster_id, q1])
    sched.add_job(random_four_news, 'cron', hour='7', id="s2", args=[df, cluster_id, q1])
    sched.add_job(random_four_news, 'cron', hour='13', id="s3", args=[df, cluster_id, q1])
    sched.add_job(random_four_news, 'cron', hour='16', id="s4", args=[df, cluster_id, q1])
    sched.add_job(random_four_news, 'cron', hour='19', id="s5", args=[df, cluster_id, q1])
    sched.add_job(random_four_news, 'cron', hour='22', id="s6", args=[df, cluster_id, q1])
    process_four = Process(target=random_four_news, args=(q1,))
    process_summary = Process(target=summary, args=(q1,q2,))
    process_mlheadline = Process(target=ml_headline, args=(q2,"test",))
    
    process_four.start()
    process_summary.start()
    process_mlheadline.start()
    
    q1.close()
    q2.close()
    q1.join_thread()
    
    process_four.join()
    process_summary.join()
    process_mlheadline.join()